In [ ]:
# imports (a lot unnecessary)
import pandas as pd
import numpy as np 
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
from google_drive_downloader import GoogleDriveDownloader as gdd
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import linkage
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# example for getting the csv's into pandas
unclean_species_df = pd.read_csv('/content/drive/MyDrive/CIS 450/species(unclean).csv')
unclean_species_df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119243,ZION-2791,Zion National Park,Vascular Plant,Solanales,Solanaceae,Solanum triflorum,Cut-Leaf Nightshade,Approved,Present,Native,Uncommon,NaN,NaN,NaN
119244,ZION-2792,Zion National Park,Vascular Plant,Vitales,Vitaceae,Vitis arizonica,Canyon Grape,Approved,Present,Native,Uncommon,NaN,NaN,NaN
119245,ZION-2793,Zion National Park,Vascular Plant,Vitales,Vitaceae,Vitis vinifera,Wine Grape,Approved,Present,Not Native,Uncommon,NaN,NaN,NaN
119246,ZION-2794,Zion National Park,Vascular Plant,Zygophyllales,Zygophyllaceae,Larrea tridentata,Creosote Bush,Approved,Present,Native,Rare,NaN,NaN,NaN


In [ ]:
unclean_species_df = pd.read_csv('/content/drive/MyDrive/CIS 450/species(unclean).csv')
unclean_species_df['Common Names'] = unclean_species_df['Common Names'].str.split(", ").tolist() #make it a list in the column val

#for keeping unclean_species_df removed
clean_species_df = unclean_species_df

#create split dataframe
split_df = pd.DataFrame(clean_species_df['Common Names'].tolist(), columns=['CN1', 'CN2', 'CN3', 'CN4', 'CN5', 'CN6', 'CN7', 'CN8', 'CN9', 'CN10', 'CN11', 'CN12', 'CN13', 'CN14']) 

# concat df and split_df
new_df = pd.concat([clean_species_df, split_df], axis=1)
new_df


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,...,CN5,CN6,CN7,CN8,CN9,CN10,CN11,CN12,CN13,CN14
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,[Moose],Approved,Present,Native,...,None,None,None,None,None,None,None,None,None,None
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"[Northern White-Tailed Deer, Virginia Deer, Wh...",Approved,Present,Native,...,None,None,None,None,None,None,None,None,None,None
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"[Coyote, Eastern Coyote]",Approved,Present,Not Native,...,None,None,None,None,None,None,None,None,None,None
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"[Eastern Timber Wolf, Gray Wolf, Timber Wolf]",Approved,Not Confirmed,Native,...,None,None,None,None,None,None,None,None,None,None
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"[Black Fox, Cross Fox, Eastern Red Fox, Fox, R...",Approved,Present,Unknown,...,Red Fox,Silver Fox,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119243,ZION-2791,Zion National Park,Vascular Plant,Solanales,Solanaceae,Solanum triflorum,[Cut-Leaf Nightshade],Approved,Present,Native,...,None,None,None,None,None,None,None,None,None,None
119244,ZION-2792,Zion National Park,Vascular Plant,Vitales,Vitaceae,Vitis arizonica,[Canyon Grape],Approved,Present,Native,...,None,None,None,None,None,None,None,None,None,None
119245,ZION-2793,Zion National Park,Vascular Plant,Vitales,Vitaceae,Vitis vinifera,[Wine Grape],Approved,Present,Not Native,...,None,None,None,None,None,None,None,None,None,None
119246,ZION-2794,Zion National Park,Vascular Plant,Zygophyllales,Zygophyllaceae,Larrea tridentata,[Creosote Bush],Approved,Present,Native,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
new_new_df = new_df.assign(common_name=new_df['Common Names']).explode('common_name')

species_to_common_name = new_new_df[['common_name','Species ID']]
species_to_common_name1 = species_to_common_name.apply(lambda s:s.str.replace('\'', ""))
species_to_common_name2 = species_to_common_name1.apply(lambda s:s.str.replace('"', ""))
species_to_common_name3 = species_to_common_name2.apply(lambda s:s.str.replace('`', ""))
species_to_common_name3['CommonName'] = species_to_common_name3['common_name'].apply(lambda s:s.replace('-', " "))
species_to_common_name3['CommonName'] = species_to_common_name3['common_name'].apply(lambda s:s.replace('_', ""))
species_to_common_name3['nameId'] = np.arange(len(species_to_common_name3))
species_to_common_name3['SpeciesId'] = species_to_common_name3['Species ID']
species_to_common_name4 = species_to_common_name3[['nameId', 'CommonName','SpeciesId']]
#df.loc[df['col1'] == value]
# my_df = species_to_common_name4.loc[species_to_common_name4['common_name'] == 'Moose']
species_to_common_name4

,nameId,CommonName,SpeciesId
0,0,Moose,ACAD-1000
1,1,Northern White-Tailed Deer,ACAD-1001
1,2,Virginia Deer,ACAD-1001
1,3,White-Tailed Deer,ACAD-1001
2,4,Coyote,ACAD-1002
...,...,...,...
119243,156187,Cut-Leaf Nightshade,ZION-2791
119244,156188,Canyon Grape,ZION-2792
119245,156189,Wine Grape,ZION-2793
119246,156190,Creosote Bush,ZION-2794


In [ ]:
from google.colab import files

species_to_common_name4.to_csv('CommonName_To_Species.csv', index=False, encoding = 'utf-8-sig')
files.download('CommonName_To_Species.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
remove_dupes_species = pd.read_csv('/content/CommonName_To_Species.csv')
#only remove dashes from common_name column

#uppercase common_name into a new column
uppercase_list = remove_dupes_species['common_name'].apply(lambda x: x.upper())
remove_dupes_species['uppercaseNames'] = uppercase_list
#remove dupes based on matching uppercased common_names
remove_dupes_species = remove_dupes_species.drop_duplicates(subset=['uppercaseNames'])
remove_dupes_species = remove_dupes_species[['common_name', 'Species ID']]
remove_dupes_species

,common_name,Species ID
0,Moose,ACAD-1000
1,Northern WhiteTailed Deer,ACAD-1001
2,Virginia Deer,ACAD-1001
3,WhiteTailed Deer,ACAD-1001
4,Coyote,ACAD-1002
...,...,...
38636,Southwestern Dwarf Mistletoe,ZION-2744
38639,Utah SpikeMoss,ZION-2770
38641,Longstalk Bindweed,ZION-2772
38642,Smalltooth Dodder,ZION-2774


In [ ]:
#downloading common name to species after duplicate cleaning
from google.colab import files

remove_dupes_species.to_csv('CommonName_To_Species_No_Dupes.csv', index=True, encoding = 'utf-8-sig')
files.download('CommonName_To_Species_No_Dupes.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#cleans up the species csv
newly_cleaned_df =  pd.read_csv('/content/species(unclean).csv') # get the csv file in 
newly_cleaned_df = newly_cleaned_df.drop(columns=['Common Names'])
newly_cleaned_df.Seasonality = newly_cleaned_df.Seasonality.fillna('Year Round')
newly_cleaned_df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Species ID,Park Name,Category,Order,Family,Scientific Name,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,Approved,Present,Native,Abundant,Year Round,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,Approved,Present,Not Native,Common,Year Round,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,Approved,Not Confirmed,Native,NaN,Year Round,Endangered,NaN
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,Approved,Present,Unknown,Common,Breeder,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119243,ZION-2791,Zion National Park,Vascular Plant,Solanales,Solanaceae,Solanum triflorum,Approved,Present,Native,Uncommon,Year Round,NaN,NaN
119244,ZION-2792,Zion National Park,Vascular Plant,Vitales,Vitaceae,Vitis arizonica,Approved,Present,Native,Uncommon,Year Round,NaN,NaN
119245,ZION-2793,Zion National Park,Vascular Plant,Vitales,Vitaceae,Vitis vinifera,Approved,Present,Not Native,Uncommon,Year Round,NaN,NaN
119246,ZION-2794,Zion National Park,Vascular Plant,Zygophyllales,Zygophyllaceae,Larrea tridentata,Approved,Present,Native,Rare,Year Round,NaN,NaN


In [ ]:
from google.colab import files
# replace park name with park code
parks_df =  pd.read_csv('/content/parks.csv') # get the csv file in

# Merge the dataframes into another dataframe based on PERSONID and Badge_ID
merged_df = pd.merge(newly_cleaned_df, parks_df, how='outer', on=['Park Name', 'Park Name'])
merged_df = merged_df.drop(columns=['Park Name', 'State', 'Acres', 'Latitude', 'Longitude', 'Unnamed: 13'])
merged_df
merged_df.to_csv('Finalized_Species.csv', index=False, encoding = 'utf-8-sig')
files.download('Finalized_Species.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# working on the weather df
weather_df =  pd.read_csv('/content/WeatherEvents_Jan2016-Dec2021.csv') # get the csv file in


FileNotFoundError: ignored

In [ ]:
weather_df['ZipCode'] = weather_df['ZipCode'].fillna(0)
weather_df['ZipCode'] = weather_df['ZipCode'].astype(int)
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7479165 entries, 0 to 7479164
Data columns (total 14 columns):
 #   Column             Dtype  
---  ------             -----  
 0   EventId            object 
 1   Type               object 
 2   Severity           object 
 3   StartTime(UTC)     object 
 4   EndTime(UTC)       object 
 5   Precipitation(in)  float64
 6   TimeZone           object 
 7   AirportCode        object 
 8   LocationLat        float64
 9   LocationLng        float64
 10  City               object 
 11  County             object 
 12  State              object 
 13  ZipCode            int64  
dtypes: float64(3), int64(1), object(10)
memory usage: 798.9+ MB


In [ ]:
weather_df.describe()

,Precipitation(in),LocationLat,LocationLng,ZipCode
count,7.479165e+06,7.479165e+06,7.479165e+06,7.419931e+06
mean,9.518492e-02,3.873336e+01,-9.192213e+01,5.247809e+04
std,9.185906e-01,5.464815e+00,1.347916e+01,2.571490e+04
min,0.000000e+00,2.455710e+01,-1.245550e+02,1.022000e+03
25%,0.000000e+00,3.449780e+01,-9.782830e+01,3.131400e+04
50%,0.000000e+00,3.927500e+01,-8.982220e+01,5.401700e+04
75%,5.000000e-02,4.295500e+01,-8.195650e+01,7.352100e+04
max,1.104130e+03,4.894020e+01,-6.779280e+01,9.936200e+04


In [ ]:
weather_df.head(20)

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [ ]:
weather_df.tail()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
7479160,W-7577193,Storm,Severe,2021-12-26 18:19:00,2021-12-26 18:29:00,0.00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479161,W-7577194,Snow,Light,2021-12-26 18:29:00,2021-12-26 18:38:00,0.02,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479162,W-7577195,Snow,Light,2021-12-28 00:53:00,2021-12-28 02:27:00,0.08,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479163,W-7577196,Snow,Moderate,2021-12-28 02:27:00,2021-12-28 03:09:00,0.09,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479164,W-7577197,Snow,Light,2021-12-28 03:09:00,2021-12-28 03:19:00,0.01,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0


In [ ]:
zips = weather_df.ZipCode.unique()

#sort values smallest to largest
zips.sort()

# create new df for all unique zips
unique_zips_df =  pd.DataFrame(zips, columns=['ZipCode'])
unique_zips_df


,ZipCode
0,0
1,1022
2,1085
3,1201
4,1247
...,...
2016,99163
2017,99212
2018,99224
2019,99301


In [ ]:
unique_zips_df.to_csv('Unique_Zip_Codes.csv', encoding = 'utf-8-sig')
files.download('Unique_Zip_Codes.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
new_weather_df = weather_df
new_weather_df['StartTime(UTC)'] = pd.to_datetime(new_weather_df['StartTime(UTC)'])
new_weather_df['EndTime(UTC)'] = pd.to_datetime(new_weather_df['EndTime(UTC)'])
new_weather_df['Duration_in_mins'] = new_weather_df['EndTime(UTC)'] - new_weather_df['StartTime(UTC)']
new_weather_df['Duration_in_mins'] = new_weather_df['Duration_in_mins']/np.timedelta64(1,'m')
new_weather_df

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,Duration,Duration_in_mins
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,0 days 01:20:00,80.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,0 days 00:40:00,40.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,0 days 09:40:00,580.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,0 days 00:20:00,20.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,0 days 02:00:00,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7479160,W-7577193,Storm,Severe,2021-12-26 18:19:00,2021-12-26 18:29:00,0.00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520,0 days 00:10:00,10.0
7479161,W-7577194,Snow,Light,2021-12-26 18:29:00,2021-12-26 18:38:00,0.02,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520,0 days 00:09:00,9.0
7479162,W-7577195,Snow,Light,2021-12-28 00:53:00,2021-12-28 02:27:00,0.08,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520,0 days 01:34:00,94.0
7479163,W-7577196,Snow,Moderate,2021-12-28 02:27:00,2021-12-28 03:09:00,0.09,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520,0 days 00:42:00,42.0


In [ ]:
new_weather_df.isna().sum()

EventId                  0
Type                     0
Severity                 0
StartTime(UTC)           0
EndTime(UTC)             0
Precipitation(in)        0
TimeZone                 0
AirportCode              0
LocationLat              0
LocationLng              0
City                 14563
County                   0
State                    0
ZipCode                  0
Duration                 0
Duration_in_mins         0
dtype: int64

In [ ]:
#!zip -r '/content/folder/"NewFileName.zip"' '/content/folder/"OrginalFileName"'

new_weather_df.to_csv('Cleaned_Weather_Events.csv', encoding = 'utf-8-sig')
#files.download('Unique_Zip_Codes.csv')

In [ ]:
!zip -r '/content/Cleaned_Weather_Events.csv.zip' '/content/Cleaned_Weather_Events.csv'

  adding: content/Cleaned_Weather_Events.csv (deflated 89%)


In [ ]:
files.download('Cleaned_Weather_Events.csv.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Removing the index column in the csv
weather_df_noIndex =  pd.read_csv('/content/Cleaned_Weather_Events.csv', index_col=False).drop(columns=['Unnamed: 0', 'TimeZone', 'AirportCode', 'City', 'County']) # get the csv file in
weather_df_noIndex

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),LocationLat,LocationLng,State,ZipCode,Duration,Duration_in_mins
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,38.0972,-106.1689,CO,81149,0 days 01:20:00,80.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,38.0972,-106.1689,CO,81149,0 days 00:40:00,40.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,38.0972,-106.1689,CO,81149,0 days 09:40:00,580.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,38.0972,-106.1689,CO,81149,0 days 00:20:00,20.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,38.0972,-106.1689,CO,81149,0 days 02:00:00,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7479160,W-7577193,Storm,Severe,2021-12-26 18:19:00,2021-12-26 18:29:00,0.00,42.5833,-108.2833,WY,82520,0 days 00:10:00,10.0
7479161,W-7577194,Snow,Light,2021-12-26 18:29:00,2021-12-26 18:38:00,0.02,42.5833,-108.2833,WY,82520,0 days 00:09:00,9.0
7479162,W-7577195,Snow,Light,2021-12-28 00:53:00,2021-12-28 02:27:00,0.08,42.5833,-108.2833,WY,82520,0 days 01:34:00,94.0
7479163,W-7577196,Snow,Moderate,2021-12-28 02:27:00,2021-12-28 03:09:00,0.09,42.5833,-108.2833,WY,82520,0 days 00:42:00,42.0


In [ ]:
from google.colab import files
weather_df_noIndex.to_csv('Final_Cleaned_Weather_Events.csv', encoding = 'utf-8-sig', index=False)
!zip -r '/content/Final_Cleaned_Weather_Events.csv.zip' '/content/Final_Cleaned_Weather_Events.csv'
files.download('Final_Cleaned_Weather_Events.csv.zip')

updating: content/Final_Cleaned_Weather_Events.csv (deflated 88%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Scraping for park wikipedia images
!pip install wptools
!pip install wikipedia
!pip install wordcloud
import pandas as pd
import numpy as np 
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
from google_drive_downloader import GoogleDriveDownloader as gdd
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import linkage

     |████████████████████████████████| 233 kB 8.6 MB/s 
  Created wheel for pycurl: filename=pycurl-7.45.1-cp37-cp37m-linux_x86_64.whl size=300939 sha256=f60e5bda1020e775bc3236ec7fba56e66d1effef9a7bcc30fc512048347d7058
  Stored in directory: /root/.cache/pip/wheels/b4/fe/1f/e6cae4153dbc082271bd5763ad36ae255639e4641054db10bc
Successfully built pycurl
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=22b829241e7aaa9e8669bb9614080820ede0757c8faeb94ef7673a51dc55aa19
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [ ]:
#web scraping for parks
import wikipedia
import requests
import json
first_parks_df = pd.read_csv('/content/parks.csv')
parksNames = first_parks_df['Park Name'].tolist()
parkImages = [];



In [ ]:
WIKI_REQUEST = 'http://en.wikipedia.org/w/api.php?action=query&prop=pageimages&format=json&piprop=original&titles='

def get_wiki_image(search_term):
    try:
        result = wikipedia.search(search_term, results = 1)
        wikipedia.set_lang('en')
        wkpage = wikipedia.WikipediaPage(title = result[0])
        title = wkpage.title
        response  = requests.get(WIKI_REQUEST+title)
        json_data = json.loads(response.text)
        img_link = list(json_data['query']['pages'].values())[0]['original']['source']
        return img_link        
    except:
        return 0

for i in (parksNames):
  parkImages.append(get_wiki_image(i))



['https://upload.wikimedia.org/wikipedia/commons/7/76/Bass_Harbor_Lighthouse_b.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/f/f0/Delicate_arch_sunset.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/b/b9/MK00609_Badlands.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/f/f6/Canyon%2C_Rio_Grande%2C_Texas.jpeg',
 'https://upload.wikimedia.org/wikipedia/commons/4/48/Biscayne.JPG',
 'https://upload.wikimedia.org/wikipedia/commons/0/0b/Black_Canyon_and_Gunnison_River_2008.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/c/c3/Inspiration_Point_Bryce_Canyon_November_2018_panorama.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/9/99/Green_River_Overlook_Ekker_Butte.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/1/1f/Capitol_Reef_National_Park.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/d/dd/Carlsbad_Interior_Formations.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/4/42/Channel_Islands_National_Park_by_Sentinel-2.jpg',
 'https://upl

In [ ]:
first_parks_df['ParkImageURL'] = parkImages
from google.colab import files
first_parks_df.to_csv('parksWithImageURLS.csv', encoding = 'utf-8-sig', index=False)
files.download('parksWithImageURLS.csv')

NameError: ignored

In [ ]:
#Web scraping for species images
import pandas as pd
import numpy as np
!pip install wikipedia
import wikipedia
import requests
import json

species_df = pd.read_csv('/content/drive/MyDrive/CIS 450/Finalized_Species.csv')


,ParkID
count,119248.000000
mean,28.916317
std,15.661530
min,0.000000
25%,15.000000
50%,28.000000
75%,44.000000
max,55.000000


In [ ]:
species_df

,Species ID,Category,Order,Family,Scientific Name,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,ParkID,Park Code
0,ACAD-1000,Mammal,Artiodactyla,Cervidae,Alces alces,Approved,Present,Native,Rare,Resident,NaN,0,ACAD
1,ACAD-1001,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,Approved,Present,Native,Abundant,Year Round,NaN,0,ACAD
2,ACAD-1002,Mammal,Carnivora,Canidae,Canis latrans,Approved,Present,Not Native,Common,Year Round,Species of Concern,0,ACAD
3,ACAD-1003,Mammal,Carnivora,Canidae,Canis lupus,Approved,Not Confirmed,Native,NaN,Year Round,Endangered,0,ACAD
4,ACAD-1004,Mammal,Carnivora,Canidae,Vulpes vulpes,Approved,Present,Unknown,Common,Breeder,NaN,0,ACAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119243,ZION-2791,Vascular Plant,Solanales,Solanaceae,Solanum triflorum,Approved,Present,Native,Uncommon,Year Round,NaN,55,ZION
119244,ZION-2792,Vascular Plant,Vitales,Vitaceae,Vitis arizonica,Approved,Present,Native,Uncommon,Year Round,NaN,55,ZION
119245,ZION-2793,Vascular Plant,Vitales,Vitaceae,Vitis vinifera,Approved,Present,Not Native,Uncommon,Year Round,NaN,55,ZION
119246,ZION-2794,Vascular Plant,Zygophyllales,Zygophyllaceae,Larrea tridentata,Approved,Present,Native,Rare,Year Round,NaN,55,ZION


In [ ]:
species_df_chunk = species_df.head(100)
speciesNames = species_df_chunk['Scientific Name'].tolist()
speciesNames

['Alces alces',
 'Odocoileus virginianus',
 'Canis latrans',
 'Canis lupus',
 'Vulpes vulpes',
 'Lynx canadensis',
 'Lynx rufus',
 'Mephitis mephitis',
 'Lutra canadensis',
 'Martes pennanti',
 'Mustela',
 'Mustela erminea',
 'Mustela frenata',
 'Mustela macrodon',
 'Mustela vison',
 'Odobenus rosmarus',
 'Halichoerus grypus',
 'Phoca vitulina',
 'Procyon lotor',
 'Ursus americanus',
 'Eptesicus fuscus',
 'Lasionycteris noctivagans',
 'Lasiurus borealis',
 'Lasiurus cinereus',
 'Myotis keenii',
 'Myotis leibii',
 'Myotis lucifugus',
 'Pipistrellus subflavus',
 'Lepus americanus',
 'Castor canadensis',
 'Clethrionomys gapperi',
 'Microtus pennsylvanicus',
 'Ondatra zibethicus',
 'Peromyscus leucopus',
 'Peromyscus maniculatus',
 'Synaptomys cooperi',
 'Napaeozapus insignis',
 'Zapus hudsonius',
 'Erethizon dorsatum',
 'Mus musculus',
 'Rattus norvegicus',
 'Glaucomys',
 'Glaucomys sabrinus',
 'Glaucomys volans',
 'Marmota monax',
 'Sciurus carolinensis',
 'Tamias striatus',
 'Tamiasciur

In [ ]:
#get the images

WIKI_REQUEST = 'http://en.wikipedia.org/w/api.php?action=query&prop=pageimages&format=json&piprop=original&titles='

def get_wiki_image(search_term):
    try:
        result = wikipedia.search(search_term, results = 1)
        wikipedia.set_lang('en')
        wkpage = wikipedia.WikipediaPage(title = result[0])
        title = wkpage.title
        response  = requests.get(WIKI_REQUEST+title)
        json_data = json.loads(response.text)
        img_link = list(json_data['query']['pages'].values())[0]['original']['source']
        return img_link        
    except:
        return 0

In [ ]:
import numpy as np
import pandas as pds
def chunker(seq, size):
      for pos in range(0, len(seq), size):
          yield seq.iloc[pos:pos + size] 

chunk_size = 10
allSpeciesNames = species_df['Scientific Name']
speciesImages = []
for i in chunker(allSpeciesNames, chunk_size):
  for j in (i):
    speciesImages.append(get_wiki_image(j))



NameError: ignored